In [0]:
!pip install tflearn

     |████████████████████████████████| 102kB 4.0MB/s 
  Created wheel for tflearn: filename=tflearn-0.3.2-cp36-none-any.whl size=128208 sha256=8ce9e1718511c63a441548d4660cf854305229108b5c4362cb890cd1df45ede3
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [0]:
pip install tensorflow==1.13.2

     |████████████████████████████████| 92.6MB 47kB/s 
     |████████████████████████████████| 3.2MB 52.1MB/s 
     |████████████████████████████████| 368kB 50.0MB/s 
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [0]:
import nltk 
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents": [\r\n        {"tag": "greeting",\r\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],\r\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "goodbye",\r\n         "patterns": ["Bye", "See you later", "Goodbye"],\r\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\r\n        },\r\n        {"tag": "thanks",\r\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\r\n         "responses": ["Happy to help!", "Any time!", "My pleasure"]\r\n        },\r\n        {"tag": "hours",\r\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\r\n         "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\r\n        },\r\n        {"tag": "location",\r\n         "patterns": 

In [0]:
with open('intents.json') as json_data:
  intents=json.load(json_data)

In [0]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day'],
   'tag': 'hours'},
  {'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of London Alley and Bridge A

In [0]:
words = []
classes = []
documents = []
ignore = ['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        w = nltk.word_tokenize(pattern)
      
        words.extend(w)
        
        documents.append((w, intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [0]:
words=[stemmer.stem(w.lower())for w in words if w not in ignore]
words=sorted(list(set(classes)))
print(len(documents),"documents")
print(len(classes),"classes",classes)
print(len(words),"unique stemmed words",words)

31 documents
9 classes ['greeting', 'goodbye', 'thanks', 'hours', 'location', 'payments', 'todaysmenu', 'deliveryoption', 'menu']
9 unique stemmed words ['deliveryoption', 'goodbye', 'greeting', 'hours', 'location', 'menu', 'payments', 'thanks', 'todaysmenu']


In [0]:
#create training data
training=[]
output=[]

output_empty=[0]*len(classes)

for doc in documents:

  bag=[]
  
  pattern_words=doc[0]
 
  pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]

  for w in words:
    bag.append(1) if w in  pattern_words else bag.append(0)
    
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    training.append([bag,output_row])


random.shuffle(training)
training=np.array(training)


train_x=list(training[:,0])
train_y=list(training[:,1])

In [0]:

tf.reset_default_graph()


net=tflearn.input_data(shape=[None,len(train_x[0])])
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,len(train_y[0]),activation='softmax')
net=tflearn.regression(net)

model=tflearn.DNN(net,tensorboard_dir='tflearn_logs')


model.fit(train_x,train_y,n_epoch=1000,batch_size=8,show_metric=True)
model.save('model.tflearn')

Training Step: 34999  | total loss: 2.07853 | time: 0.101s
| Adam | epoch: 1000 | loss: 2.07853 - acc: 0.2005 -- iter: 272/279
Training Step: 35000  | total loss: 2.08258 | time: 0.108s
| Adam | epoch: 1000 | loss: 2.08258 - acc: 0.2180 -- iter: 279/279
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [0]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [0]:
data=pickle.load(open("training_data","rb"))
words=data['words']
classes=data['classes']
train_x=data['train_x']
train_y=data['train_y']

In [0]:
with open('intents.json') as json_data:
  intents=json.load(json_data)

In [0]:
#load the saved model
model.load('./model.tflearn')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [0]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))



In [0]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [0]:
response('What is menu for today?')

You can check out the food menu at www.mymenu.com


In [0]:
response('What are you hours of operation?')

In [0]:
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
   
    results = model.predict([bow(sentence, words)])[0]

    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
 
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
   
    if results:
        
        while results:
            for i in intents['intents']:
               
                if i['tag'] == results[0][0]:
                   
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                   
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        
                        return print(random.choice(i['responses']))

            results.pop(0)


In [0]:

response('What is menu for today?')

Today's special is Chicken Tikka
